## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
2. Pass the STAC item into the raster API `/stac/tilejson.json` endpoint.
3. Using `folium.Map`, visualize the plumes.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data

The EMIT instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with JPL Open Science and Open Data policy.

# Installing the Required Libraries
Please run the next cell to install all the required libraries to run the notebook.

In [23]:
%pip install requests
%pip install folium
%pip install rasterstats
%pip install pystac_client

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Querying the STAC API

In [24]:
import requests
from folium import Map, TileLayer
from pystac_client import Client

In [25]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://ghg.center/api/stac"
RASTER_API_URL = "https://ghg.center/api/raster"

#Please use the collection name similar to the one used in STAC collection.

# Name of the collection for methane emission plumes. 
collection_name = "emit-ch4plume-v1"

In [26]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'emit-ch4plume-v1',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1'}],
 'title': 'Methane Point Source Plume Complexes',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-118.65756225585938,
     -38.788387298583984,
     151.0906524658203,
     50.24619674682617]]},
  'temporal': {'interval': [['2022-08-10T06:49:57+00:00',
     '2023-07-29T10:06:30+00:00']]}},
 'license': 'CC0-1.0',
 'keywords': None,
 'providers': None

Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [27]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [28]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 505 items


In [29]:
# Examining the first item in the collection
items[0]

{'id': 'EMIT_L2B_CH4PLM_001_20230729T100630_000234',
 'bbox': [61.67975744168143,
  39.96112852373608,
  61.690059859566304,
  39.97739549934377],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230729T100630_000234'}],
 'assets': {'ch4-plume-emissions': {'href': 's3://lp-prod-protected/EMITL2BCH4PLM.001/EMIT_L2B_CH4PLM_001_20230729T100630_000234/EMIT_L2B_CH4PLM_001_20230729T1006

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

## Exploring Methane Emission Plumes (CH₄) using the Raster API

In this notebook, we will explore global methane emission plumes from point sources. We will visualize the outputs on a map using `folium.`

In [30]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["id"][20:]: item for item in items} 
asset_name = "ch4-plume-emissions"

In [31]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this for only one item so that we can visualize the event. 

In [32]:
# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
item_id = "20230729T100630_000234"
color_map = "magma"
methane_plume_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[item_id]['collection']}&item={items[item_id]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
methane_plume_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://1w7hfngnp7.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=emit-ch4plume-v1&item=EMIT_L2B_CH4PLM_001_20230729T100630_000234&assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-394.7409973144531%2C1693.932861328125'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [61.67975744168143,
  39.96112852373608,
  61.690059859566304,
  39.97739549934377],
 'center': [61.68490865062387, 39.969262011539925, 0]}

## Visualizing CH₄ Emission Plume


In [33]:
# We will import folium to map and folium.plugins to allow side-by-side mapping
import folium
import folium.plugins

# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(methane_plume_tile["center"][1], methane_plume_tile["center"][0]), zoom_start=13)

# December 2001
map_layer = TileLayer(
    tiles=methane_plume_tile["tiles"][0],
    attr="GHG",
    opacity=1,
)
map_layer.add_to(map_)

# visualising the map
map_



# Calculating Zonal Statistics
To perform zonal statistics, first we need to create a polygon. In this use case we will create a polygon around the plume.

In [34]:
# Plume AOI 
plumes_coordinates = items[item_id]["geometry"]["coordinates"]
methane_plume_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates":
            plumes_coordinates,
        "type": "Polygon",
    },
}

In [35]:
# We'll plug in the coordinates for a location
# central to the study area and a reasonable zoom level

import folium
region_name = "Place_Holder" # please put the name of the place you are trying to visualize
aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        plumes_coordinates[0][0][1],
        plumes_coordinates[0][0][0]
    ],
    zoom_start=12,
)

folium.GeoJson(methane_plume_aoi, name=region_name).add_to(aoi_map)
aoi_map

In [36]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}"
).json()["features"]
print(f"Found {len(items)} items")

Found 505 items


In [37]:
# Explore the first item
items[0]

{'id': 'EMIT_L2B_CH4PLM_001_20230729T100630_000234',
 'bbox': [61.67975744168143,
  39.96112852373608,
  61.690059859566304,
  39.97739549934377],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://e6v7j4ejp6.execute-api.us-west-2.amazonaws.com/api/stac/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230729T100630_000234'}],
 'assets': {'ch4-plume-emissions': {'href': 's3://lp-prod-protected/EMITL2BCH4PLM.001/EMIT_L2B_CH4PLM_001_20230729T100630_000234/EMIT_L2B_CH4PLM_001_20230729T1006

In [38]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    print(result)
    return {
        **result["properties"],
        "item_id": item["id"][20:],
    }

In [39]:
for item in items:
    print(item["id"])
    break

EMIT_L2B_CH4PLM_001_20230729T100630_000234


With the function above, we can generate the statistics for the area of interest.

In [40]:
%%time
stats = [generate_stats(item, methane_plume_aoi) for item in items]

{'type': 'Feature', 'geometry': {'coordinates': [[[61.67975744168143, 39.96112852373608], [61.690059859566304, 39.96112852373608], [61.690059859566304, 39.97739549934377], [61.67975744168143, 39.97739549934377], [61.67975744168143, 39.96112852373608]]], 'type': 'Polygon'}, 'properties': {'statistics': {'b1': {'min': -394.7409973144531, 'max': 1693.932861328125, 'mean': 280.35348462301585, 'count': 378.0, 'sum': 105973.6171875, 'std': 345.7089519227557, 'median': 236.6074981689453, 'majority': -379.97967529296875, 'minority': -394.7409973144531, 'unique': 254.0, 'histogram': [[27.0, 61.0, 97.0, 86.0, 48.0, 38.0, 15.0, 1.0, 3.0, 2.0], [-394.7409973144531, -185.8736114501953, 22.9937744140625, 231.8611602783203, 440.7285461425781, 649.595947265625, 858.4633178710938, 1067.3306884765625, 1276.1981201171875, 1485.0654296875, 1693.932861328125]], 'valid_percent': 66.32, 'masked_pixels': 192.0, 'valid_pixels': 378.0, 'percentile_98': 1053.6788208007808, 'percentile_2': -308.2785339355469}}}}


In [41]:
stats[0]

{'statistics': {'b1': {'min': -394.7409973144531,
   'max': 1693.932861328125,
   'mean': 280.35348462301585,
   'count': 378.0,
   'sum': 105973.6171875,
   'std': 345.7089519227557,
   'median': 236.6074981689453,
   'majority': -379.97967529296875,
   'minority': -394.7409973144531,
   'unique': 254.0,
   'histogram': [[27.0, 61.0, 97.0, 86.0, 48.0, 38.0, 15.0, 1.0, 3.0, 2.0],
    [-394.7409973144531,
     -185.8736114501953,
     22.9937744140625,
     231.8611602783203,
     440.7285461425781,
     649.595947265625,
     858.4633178710938,
     1067.3306884765625,
     1276.1981201171875,
     1485.0654296875,
     1693.932861328125]],
   'valid_percent': 66.32,
   'masked_pixels': 192.0,
   'valid_pixels': 378.0,
   'percentile_98': 1053.6788208007808,
   'percentile_2': -308.2785339355469}},
 'item_id': '20230729T100630_000234'}

In [42]:
import pandas as pd


def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    # df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df.head(1)

,item_id,min,max,mean,count,sum,std,median,majority,minority,unique,histogram,valid_percent,masked_pixels,valid_pixels,percentile_98,percentile_2
0,20230729T100630_000234,-394.740997,1693.932861,280.353485,378.0,105973.617188,345.708952,236.607498,-379.979675,-394.740997,254.0,"[[27.0, 61.0, 97.0, 86.0, 48.0, 38.0, 15.0, 1....",66.32,192.0,378.0,1053.678821,-308.278534


In [43]:
plume_tile_2 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[0]['collection']}&item={items[0]['id']}"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
plume_tile_2

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://1w7hfngnp7.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=emit-ch4plume-v1&item=EMIT_L2B_CH4PLM_001_20230729T100630_000234&assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-394.7409973144531%2C1693.932861328125'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [61.67975744168143,
  39.96112852373608,
  61.690059859566304,
  39.97739549934377],
 'center': [61.68490865062387, 39.969262011539925, 0]}

In [45]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium
plume_tile_2_coordinates = items[0]["geometry"]["coordinates"]
aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        plume_tile_2_coordinates[0][0][1],
        plume_tile_2_coordinates[0][0][0]
    ],
    zoom_start=13,
)

map_layer = TileLayer(
    tiles=plume_tile_2["tiles"][0],
    attr="GHG", opacity = 1
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary


In this notebook we have successfully explored, analyzed, and visualized the STAC collection for EMIT methane emission plumes.